# **DeOldify - Colorea tus fotos gracias a la inteligencia artificial** 

DeOldify es una herramienta que colorea fotos en blanco y negro antiguas que hayan sido digitalizadas. A dia de hoy, DeOldify representa el estado del arte en esta tarea y gracias a este Colab, podremos colorear tantas fotos como queramos.

Esta herramienta fue desarrollada por Jason Antic, que entrenó una IA basada en una arquitectura GAN llamada NoGan. Para ello, entrenó la red mediante una coleccion de fotos a color y sus correspondientes versiones en blanco y negro. El resultado es que la red ha aprendido a colorear fotos en blanco y negro; completando la informacion que falta. 

Mientras las fotos en blanco y negro son una combinacion de pixeles en escala de grises que pueden tomar un valor entre 0-255; en las fotos a color cada pixel es una combinacion de tres valores, rojo, verde y azul donde cada uno de ellos puede tomar un valor de 0 a 255. Por tanto, la red NoGan tiene que conseguir triplicar la informacion que tiene de la foto para conseguir obtener una foto a color. 

**Autor del colab:** 
[Daniel Verdu Torres](https://davertor.github.io/)

**Referencias:**

[DeOldify](https://deoldify.ai/)

[DeOldify original github](https://github.com/jantic/DeOldify)





---


## Configurar correctamente el notebook

**<font color='#FF000'> IMPORTANTE </font>**

Primero hay que asegurarse que el entorno está configurado correctamente. Para ello, clicar en Entorno de ejecución > Cambiar entorno de ejecución y seleccionar:

* Acelerador por Hardware = GPU 


Para verificar de que hardware nos proporciona Google, ejecutar la siguiente celda.

In [ ]:
!nvidia-smi

Sun Sep 27 13:52:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Sincronizar con Google Drive

Todo lo que descarguemos y ejecutemos en Google Colab se borrará cuando cerremos la pestaña. Para no perder todos los avances y resultados, una opción recomendable suele ser conectar el notebook con nuestra cuenta de Google Drive. 

Para ello ejecutar la celda que se muestra a continuación y seguir las instrucciones. Una vez ejecutado, podemos ver a la izquierda, que una nueva carpeta ha aparecido en el árbol de carpetas.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Clonar GIT DeOldify

Ahora ya solo nos queda descargar el proyecto original de J.Antic desde su github para poder colorear nuestras fotos.


In [ ]:
!git clone https://github.com/jantic/DeOldify.git DeOldify 

Cloning into 'DeOldify'...
remote: Enumerating objects: 2129, done.
remote: Total 2129 (delta 0), reused 0 (delta 0), pack-reused 2129
Receiving objects: 100% (2129/2129), 69.36 MiB | 20.93 MiB/s, done.
Resolving deltas: 100% (965/965), done.


In [ ]:
cd DeOldify

/content/DeOldify


## Importar las librerias correspondientes

In [ ]:

from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
    print('GPU not available.')

In [ ]:
!pip install -r colab_requirements.txt

     |████████████████████████████████| 215kB 6.6MB/s 
     |████████████████████████████████| 133kB 18.0MB/s 
     |████████████████████████████████| 1.8MB 14.6MB/s 
     |████████████████████████████████| 7.8MB 27.5MB/s 
     |████████████████████████████████| 81kB 9.5MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
  Created wheel for ffmpeg: filename=ffmpeg-1.4-cp36-none-any.whl size=6083 sha256=c54682061c7d855bdae7a6d8c3b5dc029798e64e0338da6384b7635d5d63be16
  Stored in directory: /root/.cache/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
  Created wheel for typing: filename=typing-3.7.4.3-cp36-none-any.whl size=26307 sha256=1b98359268cbc5692cfa4c7e642914264f82b77062385251b5f38874a0c39e92
  Stored in directory: /root/.cache/pip/wheels/2d/04/41/8e1836e79581989c22eebac3f4e70aaac9af07b0908da173be
Successfully built ffmpeg typing
ERROR: nbclient 0.5.0 has requirement jupyter-client>=6.1.5, but you'll have jupyter-client 5.3.5 which is incompatible.


In [ ]:
import fastai
from deoldify.visualize import *

## Seleccionar el modo de colorear deseado

DeOldify incorpora dos modos diferentes para colorear las imagenes, llamados modo artistico y modo estable. Para utilizar uno u otro, se hace mediante el argumento de la funcion "artistic" en True o False. Se recomienda probar ambos y ver las diferencias.

In [ ]:
!mkdir 'models'

# Para ejecutar el modo artistico, descomentar la siguiente celda e inicializar el valor artistic = True
!wget https://www.dropbox.com/s/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth?dl=0 -O ./models/ColorizeArtistic_gen.pth

# Para usar el modo de color estable, desscomentar lo siguiente y poner artistic = False
!wget https://www.dropbox.com/s/mwjep3vyqk5mkjc/ColorizeStable_gen.pth?dl=0 -O ./models/ColorizeStable_gen.pth

--2020-09-27 13:52:41--  https://www.dropbox.com/s/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.1, 2620:100:6023:1::a27d:4301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth [following]
--2020-09-27 13:52:41--  https://www.dropbox.com/s/raw/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5ddeee1db860d2fa992ea7ecc5.dl.dropboxusercontent.com/cd/0/inline/BANH8NY0vKtNXy0MOGnGXV9hB68R_CEEhKdKHC-M00aeMTmBFFhSo7qQlRH7bCI09Mdssy9ize9sK4NqYUBQbEevzWu8Jb7ygtBVYM9Ew0GoNg/file# [following]
--2020-09-27 13:52:41--  https://uc5ddeee1db860d2fa992ea7ecc5.dl.dropboxusercontent.com/cd/0/inline/BANH8NY0vKtNXy0MOGnGXV9hB68R_CEEhKdKHC-M00aeMTmBFFhSo7qQlRH7bCI09Mdssy9ize9sK4NqYUBQbEevzWu

## Ejecucción

Ya está todo configurado correctamente, asi que nos podemos poner a colorear nuestras fotos en blanco y negro.

### Para colorear una única imagen

In [ ]:
# Introducir la ruta de vuestras imágenes. En mi caso, la tengo almacenada en Drive asi que la ruta sería la siguiente 

img_path = "/content/drive/My Drive/test_images/Torralba/70cd4b4849-ANTIGUA 7.jpg"

In [ ]:
# Imagen original 
show_image_in_notebook(img_path)

# Poner a False para usar el modo estable, y a True si se quiere usar el modo artistico.
colorizer = get_image_colorizer(artistic=True)

# Imagen coloreada
img = colorizer.plot_transformed_image(img_path,render_factor=35, watermarked=False)

### Para colorear todas las imágenes de una carpeta

In [ ]:
from PIL import Image
import os
import glob

img_list = []
img_folder = "/content/drive/My Drive/test_images/Torralba/"
img_ext = '.jpg'

# Definir a True o False dependiendo del modo que se quiera seleccionar
colorizer = get_image_colorizer(artistic=True)
# colorizer = get_image_colorizer(artistic=False)

%cd /content/drive/My Drive/test_images/Familia/
find_pattern = '*' + img_ext
img_list = glob.glob(find_pattern)
%cd /content/DeOldify

num_img = len(img_list)

for idx, img_path in enumerate(img_list, start=1):
  print("Coloreando la imagen ", idx, " de ", num_img, " imágenes")
  img_path = os.path.join(img_folder, img_path)
  img_out_path = img_path.strip(img_ext) + "_color_artistic" + img_ext
  
  # Comprobar si la foto existe en el directorio de salida
  if os.path.isfile(img_out_path):
    print("Saltando a la próxima foto")
  else:
    img = colorizer.plot_transformed_image(img_path,render_factor=35, watermarked=False)
    img_pil = Image.open(img)
    img_pil.save(img_out_path)

print("Se ha finalizado la colorización de las imágenes. Compruebe los resultados.")
